# Student Subjectivity in Course Evaluations

In [7]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer

## Create the dataset

In [2]:
set_df = pd.read_csv('cleaned-set.csv')
tcf_df = pd.read_csv('cleaned-tcf.csv')

In [3]:
print(set_df.shape)
print(tcf_df.shape)

(614, 1)
(408, 11)


In [15]:
# Calculate sentiment analysis scores
sia = SentimentIntensityAnalyzer()

tcf_df['Compound Score'] = tcf_df['Review Text'].apply(lambda text: sia.polarity_scores(text)["compound"])
tcf_df['Positive Score'] = tcf_df['Review Text'].apply(lambda text: sia.polarity_scores(text)["pos"])
tcf_df['Neutral Score'] = tcf_df['Review Text'].apply(lambda text: sia.polarity_scores(text)["neu"])
tcf_df['Negative Score'] = tcf_df['Review Text'].apply(lambda text: sia.polarity_scores(text)["neg"])

set_df['Compound Score'] = set_df['Comments'].apply(lambda text: sia.polarity_scores(text)["compound"])
set_df['Positive Score'] = set_df['Comments'].apply(lambda text: sia.polarity_scores(text)["pos"])
set_df['Neutral Score'] = set_df['Comments'].apply(lambda text: sia.polarity_scores(text)["neu"])
set_df['Negative Score'] = set_df['Comments'].apply(lambda text: sia.polarity_scores(text)["neg"])

In [20]:
# join the data together
set_df['Source'] = 'SET'

mini_tcf_df = tcf_df[['Review Text', 'Compound Score', 'Positive Score', 'Neutral Score', 'Negative Score']]
mini_tcf_df['Source'] = 'TCF'
mini_tcf_df = mini_tcf_df.rename(columns={"Review Text":"Comments"})

df = pd.concat([set_df, mini_tcf_df])

# move Source to be the first column
source_col = df.pop('Source')
df.insert(0, 'Source', source_col)

# save dataframe to csv
#df.to_csv('student_eval_data.csv', index=False)

C:\Users\surfd\AppData\Local\Temp\ipykernel_42244\1917828977.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_tcf_df['Source'] = 'TCF'


## Exploratory Data Analysis